<a href="https://colab.research.google.com/github/alaaramada/documentsummrize/blob/main/document_summarizer2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF

import re
import fitz


pdf_path = '/content/drive/MyDrive/DocumentSummarizer/adib.pdf'
all_text_path = '/content/drive/MyDrive/DocumentSummarizer/adib2.txt'
output_txt2 = '/content/drive/MyDrive/DocumentSummarizer/adib.txt'
adib_summary = '/content/drive/MyDrive/DocumentSummarizer/adib_summary.txt'
adib_summary2 = '/content/drive/MyDrive/DocumentSummarizer/adib_summary2.txt'
title_txt = '/content/drive/MyDrive/DocumentSummarizer/adib_title.txt'


categories = {
    'Environment': ['energy usage and efficiency', 'climate change strategy', 'waste reduction', 'biodiversity loss', 'greenhouse gas emissions', 'carbon food print reduction'],
    'Social': ['fair pay and living wages', 'equal employment opportunity', 'employee benefits', 'workplace health and safety', 'community engagement', 'responsible supply chain partnerships', 'adhering to labor'],
    'Governance': ['governance', 'corporate governance', 'risk management', 'compliance', 'ethical business practices', 'avoiding conflicts of interest', 'accounting integrity and transparency']
    }

def pdf_to_text_All(pdf_path, all_text_path):
    # Open the PDF file in read-binary mode
    doc = fitz.open(pdf_path)
    unicode_chars_pattern = r'[\u201c\u201d\u2019s]'
    social_count = 0
    text =""
    with open(pdf_path, 'rb') as pdf_file:
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            blocks = page.get_text("dict")["blocks"]

            for block in blocks:
                if block['type'] == 0:  # text block
                    for line in block["lines"]:
                        for span in line["spans"]:
                            text += span["text"]
                            text += "\n"

        text = re.sub(unicode_chars_pattern, '', text)
        text = re.sub(r'\n', ' ', text)  # Remove newlines
        text = re.sub(r'\s+', ' ', text)

    with open(all_text_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)
    return text


from google.colab import files
import pandas as pd

# Upload the file
#uploaded = files.upload()
# Get the filename from the uploaded dictionary
#pdf_path = next(iter(uploaded.keys()))



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 51.6 MB/s eta 0:00:00


In [ ]:
!pip install PyPDF2
import PyPDF2

def extract_first_pages(pdf_path, title_txt, num_pages=5):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in range(min(num_pages, len(reader.pages))):
            text += reader.pages[page].extract_text()

    with open(title_txt, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)
    print(text)
    return text



In [ ]:
def split_content_by_headlines2(title_txt):
    doc = fitz.open(title_txt)
    pattern = re.compile(r'\.{2,}\s*\d+[A-Z\s]*')
    content_groups = []
    current_group = {}
    current_group["title"] = ""
    current_group["content"] = ""

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for block in blocks:
            if block['type'] == 0:  # text block
                for line in block["lines"]:
                    for span in line["spans"]:
                        text = span["text"].strip()
                        # Check if the text is in all capital letters
                        if text.isupper() and len(text) > 2 and not pattern.match(text):  # Adjust minimum length as needed
                            if current_group["content"]:
                                content_groups.append(current_group)
                                current_group = {"title": "", "content": ""}
                            current_group["title"] = text
                        elif pattern.match(text):
                            current_group["content"] += text + " "
                        else:
                            current_group["content"] += text + " "

    # Add the final group if it contains content
    if current_group["content"] or current_group["title"]:
        content_groups.append(current_group)

    # Remove the dublication if found
    seen_titles = set()
    unique_data = []
    for current_group["title"], current_group["content"] in content_groups:
        if current_group["title"] not in seen_titles:
            current_group["title"] = "last_title"
            current_group["content"] = "last_content"
            unique_data.append(content_groups)
            seen_titles.add(current_group["title"])
    print(content_groups)
    return content_groups

In [ ]:
def append_content_to_titles(text, content_groups, output_txt2, file_summary, file_summary2):
    titles = [group["title"] for group in content_groups]  # Create an array with just the titles
    title_count = len(titles)  # Count the number of titles

    catogry = ""

    lines = text.split('\n')
    concatenated_text = ' '.join(lines)


    with open(output_txt2, 'w', encoding='utf-8') as txt_file,\
         open(file_summary, 'w', encoding='utf-8') as txt_file2,\
         open(file_summary2, 'w', encoding='utf-8') as txt_file3:
        for i in range(0, len(titles) - 1):
                first_headline = titles[i]
                if first_headline == "last_title": continue
                headline_length = len(first_headline)
                start_indices = [m.start() for m in re.finditer(re.escape(first_headline), concatenated_text)]
                start_after_headline = start_indices[0] + headline_length
                concatenated_text = concatenated_text[start_after_headline:]

        #txt_file.write(concatenated_text + "\n")
        summaries = []
        Fsummaries = []
        for i in range(0, len(titles) -1):
            first_headline = titles[i]
            try:
                second_headline = titles[i+1]
            except IndexError:
                second_headline = first_headline
            except Exception as e:
                second_headline = first_headline


            print("**************\n")
            print(first_headline)
            print("\n",second_headline)
            #txt_file.write("\n " + first_headline + "."*50 + "\n")

            c = 0
            x = 0
            start_indices = [m.start() for m in re.finditer(re.escape(first_headline), concatenated_text)]
            print(start_indices)
            print(len(start_indices))
            last = len(titles) - 2
            if (i == last and second_headline == "last_title") or (i == len(titles) and first_headline == "last_title"):
                in_between = concatenated_text[start_indices[c]:]
                if not in_between.strip(): continue
                catogry = categorize_text(in_between, categories)
                if catogry != "Not_Found" or catogry != "":
                    summary = summarize_text(in_between, True)
                    Fsummary = summarize_text(summary, False)
                    summaries.append(summary)
                    Fsummaries.append(Fsummary)
                    print("Fsummaries", Fsummaries)
                    txt_file2.write(summary + "\n")
                    txt_file3.write(Fsummary + "\n")
                    print("catogry is:", catogry)
                print("content of: ", first_headline)
                txt_file.write(in_between + "\n")
                #txt_file.write("\n NEW_PRAG" + "."*50 + "\n")
                break

            if len(start_indices) == 0:
                print("Zero section for start")
                start = find_partial_matches(concatenated_text, first_headline)
                if start != 0:
                    start_indices = [start]
                    print("match start", start)
                else:
                    continue

            if first_headline == second_headline:
                continue
            else:
                end_indices = [m.start() for m in re.finditer(re.escape(second_headline), concatenated_text)]
                print(end_indices)
                print(len(end_indices))
                if len(end_indices) == 0:
                    print("Zero section for end")
                    end = find_partial_matches(concatenated_text, second_headline)
                    print(end)
                    if end != 0:
                        end_indices = [end]
                        end_indices = check_indicator(start_indices, end_indices)
                        in_between = concatenated_text[start_indices[c]:end_indices[x]]
                        if not in_between.strip(): continue
                        catogry = categorize_text(in_between, categories)
                        print(catogry)
                        if catogry != "Not_Found" or catogry != "":
                            summary = summarize_text(in_between, True)
                            Fsummary = summarize_text(summary, False)
                            summaries.append(summary)
                            Fsummaries.append(Fsummary)
                            print("Fsummaries", Fsummaries)
                            txt_file2.write(summary + "\n")
                            txt_file3.write(Fsummary + "\n")
                            print("catogry is:", catogry)
                        print("content of: ", first_headline)
                        txt_file.write(in_between + "\n")

                    else:
                        continue

                else:
                    end_indices = check_indicator(start_indices, end_indices)
                    in_between = concatenated_text[start_indices[c]:end_indices[x]]
                    if not in_between.strip(): continue
                    catogry = categorize_text(in_between, categories)
                    if catogry != "Not_Found" or catogry != "":
                        summary = summarize_text(in_between, True)
                        Fsummary = summarize_text(summary, False)
                        summaries.append(summary)
                        Fsummaries.append(Fsummary)
                        print("Fsummaries", Fsummaries)
                        txt_file2.write(summary + "\n")
                        txt_file3.write(Fsummary + "\n")
                        print("catogry is:", catogry)
                    print("content of: ", first_headline)
                    txt_file.write(in_between + "\n")


    #print(Fsummaries)
    return summaries

def check_indicator(start_indices, end_indices):
    endicator = end_indices
    if end_indices[0] < start_indices[0]:
        if len(end_indices) > 1:
            for l in range(0, len(end_indices)):
                if start_indices[0] < end_indices[l]:
                    endicator = [end_indices[l]]
    return endicator

def find_partial_matches(text, headline):
    # Split the title into words
    title_words = headline.split()
    # Convert the text to lower case for case-insensitive comparison
    text_lower = text
    # Check if any word from the title is present in the text
    start_position = 0
    for word in title_words:
        match = re.search(r'\b' + re.escape(word) + r'\b', text_lower, re.IGNORECASE)
        if match:
            start_position = match.start()
            break
        else:
            continue
    return start_position

def preprocess_text(text):
    # Convert to lowercase and remove special characters
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    return text.strip()

def categorize_text(text, categories):
    # Preprocess the text
    text = preprocess_text(text)
    # Dictionary to keep track of keyword matches for each category
    category_matches = {category: 0 for category in categories}
    # Check for keyword matches in each category
    for category, keywords in categories.items():
        for keyword in keywords:
            if re.search(r'\b' + re.escape(keyword) + r'\b', text):
                category_matches[category] += 1

    # Find the category with the highest match count
    max_category = max(category_matches, key=category_matches.get)
    max_count = category_matches[max_category]

    # Return the category with the most matches or None if no matches
    if max_count > 0:
        return max_category
    else:
        return "Not_Found"


In [ ]:
!pip install torch torchvision torchaudio
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2
!pip install transformers
!pip install numpy==1.24.3

!pip install numpy scipy scikit-learn

from transformers import pipeline
import textwrap

def summarize_text(text, bolean):
  summaries = []
  model='facebook/bart-large-cnn'
  max_length = 1024
  overlap = 100
  try:
      chunks = textwrap.wrap(text, width=max_length - overlap)
      #chunks =  split_line_into_chunks(text)
      #print(chunks)

      summaries = []
      for chunk in chunks:
          text_length = len(chunk)
          if text_length < 100:
            if text_length < 10: continue
            summary_max_length = 10
            summary_min_length = 5
          elif text_length < 1025:
            summary_max_length = 10
            summary_min_length = 5
          else:
            summary_max_length = 20
            summary_min_length = 5
          if bolean and text_length > 100:
            summary_max_length = max(1, int(text_length * 0.05))  # Ensure it's at least 1
            summary_min_length = min(1, int(text_length * 0.01))

          #print("summary_max_length", summary_max_length)
          summarizer2 = pipeline('summarization', model=model)
          summary = summarizer2(chunk, max_length=summary_max_length, min_length=summary_min_length, do_sample=False)
          summaries.append(summary)
          #print("sumarry: ", summary)

      #print("end summary loop")
      #print(summaries)
      combined_summary = " ".join([summary[0]['summary_text'] for summary in summaries])
      #print("combined_summary", combined_summary)
      return combined_summary

  except Exception as e:
      print(f"Error during summarization: {e}")
      return "no sumarize text"

#sum_text = "IB financing is increasingly geared towards a sustainable economy. 13 green financing transactions were closed, including AED1.7 billion green financing for three sustainable sewage treatment plants across Saudi Arabia. ADIB strives to create wider positive impacts for the society. This is achieved through its ethical Shari’a principles and responsible, innovative products and services that make banking more affordable and sustainable. Dubai has demonstrated one of the lowest levelized cost of electricity in the world of USD cents 1.6953 per kWh. Project involves the construction of a state-of-the-art 900MW solar PV plant, using bi-facial panels. Shuaa Energy 3 P.S.C. was structured as a c. 27-year soft mini perm financing with both conventional and Islamic tranches. The financing structure featured a set of equity bridge finances provided by local banks and also by DEWA. Jubail-3A IWP will utilize reverse osmosis (RO) technology to yield a capacity of 600,000 cubic meters a day. The project will benefit from a 25-year Water Purchase Agreement (WPA) with Saudi Water Partnership Company (SWPA) The plant is located in Jubail, Saudi Arabia Jazlah Water Desalination Company USD 481 million September 2020 Construction of Water Treatment Plant 26 Years Year 6 Istina ljara Hogan Lovells Covington Mandated Lead Arranger. ADIB has invested AED 1.8 billion in sustainable/green Sukuks. 93.6% of financing provided to corporates assets were subject to a combined social or environmental screening. A total value of USD 1.7 billion in sustainable project financing has been identified. ADIB’s Credit Risk Policy will be updated to include an ESG insert.  ADIB is dedicated to growing its reputation as a leading Islamic bank, exercising sound governance practices that focus on Shari’a compliance. Measuring portfolio carbon emissions is essential to be able to reduce them. Shari’a financing has steadily increased, amounting to AED 113 billion in 2022, a 22% increase on 2021. All employees are trained in these criteria, with a total of 104 hours delivered in 2022 (an increase of 4% on 2021) ADIB portfolio held around AED 1.8 billion of Green Sukuk in 2022. Proceeds from both were invested in projects such as renewable energy, energy efficiency, sustainable water management and green buildings. In 2022 the bank accelerated efforts against its three-year Digital Transformation Strategy. Examples of innovation initiatives in 2022 included expanding Interactive Teller Machines. Facial recognition artificial intelligence (AI) was used in 40% of account openings in 2022. The proportion of digitally active customers increased to 76%, with one new digital product and a 31.1% rise in online or mobile transactions. ADIB launched ACE, the region’s first digital command centre. Digital Excellence remains at the heart of ADIB’s 2025 strategy. The bank strives to become a digital-first financial institution. For customers, this means an even more seamless experience."
#Sum2_txt=  "ADIB strives to create wider positive impacts for the society. 13 green financing transactions were closed, including AED1.7 billion green financing for three sustainable sewage treatment plants across Saudi Arabia. ADIB has invested AED 1.8 billion in sustainable/green Sukuks. 93.6% of financing provided to corporates assets were subject to a combined social or environmental screening. A total value of USD 1.7 billion inustainable project financing has been identified. ADIB portfolio held around AED 1.8 billion of Green Sukuk in 2022. Shari’a financing has steadily increased, amounting to AED 113 billion in 2022, a 22% increase on 2021. ADIB aims to become a digital-first financial institution. For customers, this means an even more seamless experience."
#summarize_text(sum_text)

In [ ]:
!pip install wordcloud
!pip install matplotlib

from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
from PIL import Image

def word_cloud(sum_text):
    # Generate a word cloud
    file_path = "word_cloud.png"
    wordcloud = WordCloud(width=800, height=400, background_color='black', max_words=30, stopwords=STOPWORDS).generate(sum_text)
    # Display the word cloud
    plt.figure(figsize=(5, 3))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    #plt.show()

    # Save to a BytesIO object
    wordcloud.to_image().save(file_path, format='png')


    plt.savefig(file_path, format='png')
    plt.close()

    return file_path



In [ ]:
def read_as_spilit(file):
  with open(file, "r") as fp:
    str = fp.read().splitlines()

  return str

def read_as_text(file):
  with open(file, "r") as fp:
    str = fp.read()
  return str

#read_as_spilit('/content/drive/MyDrive/DocumentSummarizer/adib_summary.txt')



In [ ]:
def re_summarizing(sum_text):
  processed_text = process_text(sum_text)
  for line_chunks in processed_text:
    print(line_chunks)
    sumrized = sum_google_peg(documents2)
    print(sumrized)


def split_text_by_newlines(text):
  return text.splitlines()


def split_line_into_chunks(line):
  sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', line)
  chunks = [sentence.strip() for sentence in sentences if sentence]
  #chunks = [line[i:i+width] for i in range(0, len(line), width)]
  return chunks

def process_text(text):
  lines = split_text_by_newlines(text)
  result = [split_line_into_chunks(line) for line in lines]
  return result

#text = read_as_text('/content/drive/MyDrive/DocumentSummarizer/adib_summary.txt')
#re_summarizing(text)




In [ ]:
!pip install mglearn
import io
import sys
import mglearn

def classification(DATA):
  # Define the Vectorizer
  VECT: CountVectorizer = CountVectorizer(ngram_range=(1, 1), stop_words="english")
  # Transform the data to create the document-term matrix
  FIN = VECT.fit_transform(DATA)
  feature_names = VECT.get_feature_names_out()
  # Convert to DataFrame for inspection
  pd.DataFrame(FIN.toarray(), columns=VECT.get_feature_names_out()).head(1)
  # Fit the LDA model
  LDA: LatentDirichletAllocation = LatentDirichletAllocation(n_components=3, random_state=42)
  LDA_DTF = LDA.fit_transform(FIN)

  topics = []
  for idx, topic in enumerate(LDA.components_):
      top_features_ind = topic.argsort()[:-10 - 1:-1]
      top_features = [feature_names[i] for i in top_features_ind]
      topics.append(top_features)

  print(topics)
  predefined_categories = {
    'Environment': ['energy usage and efficiency', 'climate change strategy', 'waste reduction', 'biodiversity loss', 'greenhouse gas emissions', 'carbon food print reduction'],
    'Social': ['fair pay and living wages', 'equal employment opportunity', 'employee benefits', 'workplace health and safety', 'community engagement', 'responsible supply chain partnerships', 'adhering to labor'],
    'Governance': ['governance', 'corporate governance', 'risk management', 'compliance', 'ethical business practices', 'avoiding conflicts of interest', 'accounting integrity and transparency']
    }
  topic_category_mapping = {}
  category_scores = {}
  for category, keywords in predefined_categories.items():
    # Calculate overlap between topic words and category keywords
    score = len(set(topic) & set(keywords))
    category_scores[category] = score

  # Choose the category with the highest overlap score
  mapped_category = max(category_scores, key=category_scores.get)

  # Store the topic and its mapped category in the dictionary
  topic_category_mapping[f"Topic {idx+1}"] = {
      "top_words": topic,
      "category": mapped_category
  }

  return topic_category_mapping

def classification2(DATA):
    # Define the Vectorizer
  VECT: CountVectorizer = CountVectorizer(ngram_range=(1, 1), stop_words="english")
  # Transform the data to create the document-term matrix
  FIN = VECT.fit_transform(DATA)
  feature_names = VECT.get_feature_names_out()
  # Convert to DataFrame for inspection
  pd.DataFrame(FIN.toarray(), columns=VECT.get_feature_names_out()).head(1)
  # Fit the LDA model

  LDA: LatentDirichletAllocation = LatentDirichletAllocation(n_components=3)
  LDA_DTF = LDA.fit_transform(FIN)

  sorting = np.argsort(LDA.components_)[:, ::-1]
  features = np.array(VECT.get_feature_names_out())
  array = np.full((1, sorting.shape[1]), 1)
  array = np.concatenate((array, sorting), axis=0)

  output = io.StringIO()
  # Redirect stdout to capture print output
  original_stdout = sys.stdout
  sys.stdout = output
  try:
    # Call print_topics and capture the output
    mglearn.tools.print_topics(
        topics=range(1, 4),
        feature_names=features,
        sorting=array,
        topics_per_chunk=5,
        n_words=10
    )
  finally:
      # Restore the original stdout
      sys.stdout = original_stdout

  # Retrieve the captured text
  captured_text = output.getvalue()
  print(captured_text)
  output.close()

  return captured_text
#DATA = read_as_spilit('/content/drive/MyDrive/DocumentSummarizer/adib_summary.txt')
#topicss = classification(DATA)

#print(topicss)









In [ ]:
# Install nltk (if needed)
!pip install nltk
import nltk
# Download the stopwords corpus
nltk.download('stopwords')
# Import stopwords and test
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from nltk.stem import PorterStemmer


def normalize_text(text):
    return text.lower().strip()

def normalize_terms(terms):
  stop_words = set(stopwords.words('english'))
  stemmer = PorterStemmer()

  normalized = []
  for term in terms:
      # Convert to lowercase
      term = term.lower()
      # Remove punctuation
      term = term.translate(str.maketrans('', '', string.punctuation))
      # Remove stop words
      if term not in stop_words:
          # Stemming
          term = stemmer.stem(term)
          normalized.append(term)
  return normalized


def get_top_words_for_topics(model, feature_names, n_words=10):
    topics = model.components_
    top_words = {}
    for i, topic in enumerate(topics):
        top_indices = topic.argsort()[-n_words:][::-1]
        top_words[f"Topic {i+1}"] = [feature_names[index] for index in top_indices]
    return top_words



def get_top_word_and_normalize(documents):
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform(documents)

  lda = LatentDirichletAllocation(n_components=3, random_state=0)
  lda.fit(X)

  feature_names = vectorizer.get_feature_names_out()
  top_words_per_topic = get_top_words_for_topics(lda, feature_names)
  top_words_per_topic = {topic: normalize_terms(words) for topic, words in top_words_per_topic.items()}

  return top_words_per_topic

def get_top_topic_and_normalize(documents):
  top_words_per_topic = get_top_word_and_normalize(documents)
  topics = set()
  for x in top_words_per_topic:
    for y in top_words_per_topic[x]:
      topics.add(y)
  top_topic = topics
  top_topic_wo  = top_words_per_topic
  return topics
#documents = read_as_spilit('/content/drive/MyDrive/DocumentSummarizer/adib_summary.txt')
#print(get_top_word_and_normalize(documents))
#print(get_top_topic_and_normalize(documents))




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!pip install transformers torch
from transformers import PegasusTokenizer, PegasusForConditionalGeneration


def sum_google_peg(documents):
  topics = get_top_topic_and_normalize(documents)
  model_name = "google/pegasus-xsum"
  tokenizer = PegasusTokenizer.from_pretrained(model_name)
  model = PegasusForConditionalGeneration.from_pretrained(model_name)
  REPORT = "ESG Report"
  ORGANIZATION = "First Abudhabi Bank"
  tokens = tokenizer(" ".join(topics) + f" Report / Study Named {REPORT} By {ORGANIZATION}", return_tensors="pt", truncation=True)
  summary_ids = model.generate(tokens["input_ids"], max_length=1500, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  return summary

#documents = read_as_spilit('/content/drive/MyDrive/DocumentSummarizer/adib_summary.txt')
#sum_google_peg(documents)



In [ ]:

import nltk
from nltk.tokenize import sent_tokenize
from collections import Counter

# Download 'punkt' model
nltk.download('punkt')

documents = read_as_spilit('/content/drive/MyDrive/DocumentSummarizer/adib_summary.txt')
documents2 = read_as_text('/content/drive/MyDrive/DocumentSummarizer/adib_summary.txt')

categories = {
    'Environment': ['energy usage and efficiency', 'climate change strategy', 'waste reduction', 'biodiversity loss', 'greenhouse gas emissions', 'carbon food print reduction'],
    'Social': ['fair pay and living wages', 'equal employment opportunity', 'employee benefits', 'workplace health and safety', 'community engagement', 'responsible supply chain partnerships', 'adhering to labor'],
    'Governance': ['governance', 'corporate governance', 'risk management', 'compliance', 'ethical business practices', 'avoiding conflicts of interest', 'accounting integrity and transparency']
    }
def Bar_Chart_of_Keyword_Frequency(sum_text, keywords_main, file_name):
    file_path = file_name+'.png'
    if keywords_main == 'Environment' :
        keywords = ['energy usage and efficiency', 'climate change strategy', 'waste reduction', 'biodiversity loss', 'greenhouse gas emissions', 'carbon food print reduction']
    elif keywords_main == 'Social' :
        keywords = ['fair pay and living wages', 'equal employment opportunity', 'employee benefits', 'workplace health and safety', 'community engagement', 'responsible supply chain partnerships', 'adhering to labor']
    elif keywords_main == 'Governance' :
        keywords = ['governance', 'corporate governance', 'risk management', 'compliance', 'ethical business practices', 'avoiding conflicts of interest', 'accounting integrity and transparency']
    else:
        keywords = []
    # Tokenize the summary text and remove stopwords
    nltk.download('stopwords')
    tokens = nltk.word_tokenize(sum_text.lower())
    tokens = [word for word in tokens if word.isalpha()]  # Remove punctuation and numbers
    filtered_tokens = [word for word in tokens if word not in stopwords.words('english')]



    # Count the frequency of each word
    word_freq = Counter(filtered_tokens)
    # Plot the frequency of the top 10 words
    #most_common_words = word_freq.most_common(10)
    #words, counts = zip(*most_common_words)

    filtered_word_freq = {word: word_freq[word] for word in keywords}

    words = list(filtered_word_freq.keys())
    frequencies = list(filtered_word_freq.values())

    #if you need most common word without frequency
    #most_common_words_only = [word for word, freq in most_common_words]
    #print(most_common_words_only)

    plt.figure(figsize=(4, 2))
    plt.barh(words, frequencies, color='skyblue')

    #plt.bar(words, counts, color='skyblue')
    plt.xlabel('Frequency')
    plt.ylabel('Words')
    plt.title(keywords_main)
    plt.subplots_adjust(left=0.3, right=0.7)
    plt.tight_layout()
    #plt.show()
    # Save to a BytesIO object
    plt.savefig(file_path, format='png')
    plt.close()

    return file_path


def preprocess_text(text):
    # Convert to lowercase and remove special characters
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    return text.strip()

def categorize_text(text, categories):
    # Preprocess the text
    text = preprocess_text(text)

    # Dictionary to keep track of keyword matches for each category
    category_matches = {category: 0 for category in categories}
    # Check for keyword matches in each category
    for category, keywords in categories.items():
        for keyword in keywords:
            if re.search(r'\b' + re.escape(keyword) + r'\b', text):
                category_matches[category] += 1

    # Find the category with the highest match count
    max_category = max(category_matches, key=category_matches.get)
    max_count = category_matches[max_category]

    #print("max_category", max_category)
    #print("max_count", max_count)
    # Return the category with the most matches or None if no matches
    if max_count > 0:
        return max_category
    else:
        return "Not_Found"
def read_as_text(file):
  with open(file, "r") as fp:
    str = fp.read()
  return str

#keywords = categorize_text(documents2, categories)
#Bar_Chart_of_Keyword_Frequency(documents2,keywords)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


NameError: name 'read_as_spilit' is not defined

In [ ]:
def highlight_text(sum_text, keywords_main):
  lines = split_text_by_newlines(sum_text)
  final_sum_text = ""
  for line in lines:
    sum_text = line.lower()
    if keywords_main == 'Environment' :
        keywords = ['energy usage and efficiency', 'climate change strategy', 'waste reduction', 'biodiversity loss', 'greenhouse gas emissions', 'carbon food print reduction']
    elif keywords_main == 'Social' :
        keywords = ['fair pay and living wages', 'equal employment opportunity', 'employee benefits', 'workplace health and safety', 'community engagement', 'responsible supply chain partnerships', 'adhering to labor']
    elif keywords_main == 'Governance' :
        keywords = ['governance', 'corporate governance', 'risk management', 'compliance', 'ethical business practices', 'avoiding conflicts of interest', 'accounting integrity and transparency']
    else:
        keywords = []

    for keyword in keywords:
        sum_text = re.sub(rf'({keyword})', r'<b><font color="red">\1</font></b>', sum_text, flags=re.IGNORECASE)
    final_sum_text += sum_text +"<br/>"
  print(final_sum_text)
  return final_sum_text




In [ ]:
def cleaning(text):
  word_to_remove = "no sumarize text"
  # Remove the word
  cleaned_text = text.replace(word_to_remove, "")
  return cleaned_text

headlines_file_extract = extract_first_pages(pdf_path, title_txt, 5)
headlines = split_content_by_headlines2(title_txt)

All_document = pdf_to_text_All(pdf_path, output_txt2)
summaries = append_content_to_titles(All_document, headlines, output_txt2, adib_summary, adib_summary2)


NameError: name 'extract_first_pages' is not defined

In [ ]:
# For PDF generation
!pip install reportlab matplotlib wordcloud pillow numpy
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Image
from reportlab.platypus import Spacer
import numpy as np
import tempfile
import os
from datetime import datetime

def create_pdf(sum_text, documents, documents2, sub_documents, sub_documents2, categories):
    pdf_file = "report.pdf"
    #bar_chart_file = 'bar_chart.png'
    doc = SimpleDocTemplate(pdf_file, pagesize=letter)
    styles = getSampleStyleSheet()
    normal_style = styles["Normal"]
    title_style = styles["Heading2"]  # Use a different style for section titles
    subtitle_style = styles["Heading3"]  # Use a smaller heading style for subtitles
    elements = []
    keywords = categorize_text(documents, categories)
    # Title and Date
    title = "ESG Summary Report"
    date_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Add title
    elements.append(Paragraph(title, styles["Title"]))
    elements.append(Paragraph(f"Date: {date_str}", normal_style))
    elements.append(Spacer(1, 12))  # Add space after the title

    # Add the title for the highlighted text section
    elements.append(Paragraph("Abstract_text", title_style))
    elements.append(Spacer(1, 12))  # Add space after the subtitle

    # Combine highlighted text into one string
    combined_highlighted_text = highlight_text(sub_documents, keywords)
    introduction = sum_google_peg(documents2)

    introduction_paragraph = f'{introduction}'
    elements.append(Paragraph(introduction_paragraph, normal_style))
    elements.append(Spacer(1, 12))

    # Format the combined text to include bold and red styling
    highlighted_paragraph = f'{combined_highlighted_text}'
    elements.append(Paragraph(highlighted_paragraph, normal_style))
    elements.append(Spacer(1, 12))

    top_word = get_top_word_and_normalize(documents2)
    top_topic = get_top_topic_and_normalize(documents2)
    elements.append(Paragraph("Top Topic", title_style))
    elements.append(Spacer(1, 12))  # Add space after the subtitle
    top_topic_paragraph = f'{top_topic}'
    elements.append(Paragraph(top_topic_paragraph, normal_style))
    elements.append(Spacer(1, 12))

    elements.append(Paragraph("Top Word Per Topic", title_style))
    elements.append(Spacer(1, 12))  # Add space after the subtitle
    top_word_paragraph = f'{top_word}'
    elements.append(Paragraph(top_word_paragraph, normal_style))
    elements.append(Spacer(1, 12))

    elements.append(Paragraph("Classification Topic on Catogry Map", title_style))
    elements.append(Spacer(1, 12))  # Add space after the subtitle
    topicss = classification(documents2)
    topicss_paragraph = f'{topicss}'
    elements.append(Paragraph(topicss_paragraph, normal_style))
    elements.append(Spacer(1, 12))

    elements.append(Paragraph("Classification Topic on General", title_style))
    elements.append(Spacer(1, 12))  # Add space after the subtitle
    topicssg = classification2(documents2)
    for line in topicssg.split('\n'):
      if line.strip():  # Only add non-empty lines
          paragraph = Paragraph(line, normal_style)
          elements.append(paragraph)
    elements.append(Spacer(1, 12))

    elements.append(Paragraph("Visulaization", title_style))
    elements.append(Spacer(1, 12))  # Add space after the subtitle

    bar_chart_file1 = Bar_Chart_of_Keyword_Frequency(documents, 'Environment', 'Environment_bar_chart')
    print("output:",bar_chart_file1)
    # Add an Image
    if os.path.exists(bar_chart_file1):
        img = Image(bar_chart_file1, width=500, height=300)
        elements.append(img)
        elements.append(Spacer(1, 12))
    else:
        error_text = Paragraph("Error: Image not found.", normal_style)
        elements.append(error_text)
        elements.append(Spacer(1, 12))

    bar_chart_file2 = Bar_Chart_of_Keyword_Frequency(documents, 'Social', 'social_bar_chart')
    print("output:",bar_chart_file2)
    # Add an Image
    if os.path.exists(bar_chart_file2):
        img = Image(bar_chart_file2, width=500, height=300)
        elements.append(img)
        elements.append(Spacer(1, 12))
    else:
        error_text = Paragraph("Error: Image not found.", normal_style)
        elements.append(error_text)
        elements.append(Spacer(1, 12))

    bar_chart_file3 = Bar_Chart_of_Keyword_Frequency(documents, 'Governance', 'Governance_bar_chart')
    print("output:",bar_chart_file3)
    # Add an Image
    if os.path.exists(bar_chart_file3):
        img = Image(bar_chart_file3, width=500, height=300)
        elements.append(img)
        elements.append(Spacer(1, 12))
    else:
        error_text = Paragraph("Error: Image not found.", normal_style)
        elements.append(error_text)
        elements.append(Spacer(1, 12))

    word_cloud_file = word_cloud(documents)
    if os.path.exists(word_cloud_file):
        img = Image(word_cloud_file, width=500, height=300)
        elements.append(img)
        elements.append(Spacer(1, 12))
    else:
        error_text = Paragraph("Error: Image not found.", normal_style)
        elements.append(error_text)
        elements.append(Spacer(1, 12))



    # Build the PDF
    doc.build(elements)

# Generate the final summary
#sum_google_peg(documents)
documents = read_as_text('/content/drive/MyDrive/DocumentSummarizer/adib_summary.txt')
sub_documents = read_as_text('/content/drive/MyDrive/DocumentSummarizer/adib_summary2.txt')
documents2 = read_as_spilit('/content/drive/MyDrive/DocumentSummarizer/adib_summary.txt')
sub_documents2 = read_as_spilit('/content/drive/MyDrive/DocumentSummarizer/adib_summary2.txt')
#sum_text = summarize_text(documents)
sum_text = ""

create_pdf(sum_text, documents, documents2, sub_documents, sub_documents2, categories)




financial performance and economic impact data cover<br/>bank welcomed 5,280 new mall<br/>adib is a sharia-compl<br/>adib viion wa revamped to adib ha initiated the development of<br/>adib is committed to adhere to in 2022, five incident of non adib report zero data ecurity<br/>adib eek to create value<br/>adib aim to drive the h adib inveted aed in 2022, adib launched ace<br/>adib exit to erve a amwali is a digital i<br/>adib committed to creating a l adib will alo develop and al ghaf initiative i an ongoing<br/>in 2022, total community invet women make up 71.5% in 2022, more training wa delivered  adib divere taff include in 2022, 36 grievance were filed 568 premie check. were conducted<br/>aed 357,276,444 report provides an overview of our performance service reviewed that the gri content ilamic bank is the first paword prb is a<br/>


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[['bank', 'adib', '2022', 'esg', 'community', 'employee', 'principle', 'cutomer', 'reponible', 'year'], ['bank', 'adib', '2022', 'employee', 'esg', 'cutomer', 'year', 'gri', 'number', 'report'], ['topic', 'material', 'financial', 'acro', 'tatement', 'monetary', 'achievement', 'provides', 'overview', 'tranparent']]
topic 1       topic 2       topic 3       
--------      --------      --------      
bank          bank          bank          
adib          esg           adib          
2022          adib          2022          
employee      2022          community     
cutomer       year          value         
gri           sharia        uae           
number        compliance    aed           
2021          committee     wa            
report        ha            60            
year          report        cutomer       





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


output: Environment_bar_chart.png


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


output: social_bar_chart.png


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


output: Governance_bar_chart.png
